In [121]:
df_2018 = pd.read_csv('Data/Competition.csv').drop_duplicates()
print (len(df_2018[df_2018['last_name'] == 'Sponcil ']))
print (len(d_2018['matches'][d_2018['matches']['last_name'] == 'Sponcil']))

44
39
